In [ ]:
import spacy
import random
from spacy.util import minibatch, compounding

In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [ ]:
def train_model(data_dict, iterations):
    # Cria um modelo vazio do Spacy
    nlp = spacy.blank("pt")
    
    # Cria um ner (Named Entity Recognizer) vazio e adiciona ao pipeline do modelo
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)

    # Define as labels (entidades) para treinar o modelo
    for label in data_dict.keys():
        ner.add_label(label)
    
    # Prepara os dados de treinamento
    train_data = []
    for label, examples in data_dict.items():
        for text, annotations in examples:
            train_data.append((text, annotations))
    
    # Inicia o treinamento do modelo
    nlp.begin_training()
    for itn in range(iterations):
        # Embaralha os dados de treinamento para aumentar a diversidade
        random.shuffle(train_data)
        losses = {}
        
        # Divide os dados em lotes (batches) e atualiza o modelo em cada lote
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, losses=losses)
        
        # Imprime a perda (loss) durante o treinamento
        print("Iteration:", itn+1, "Loss:", losses)

    # # Salvando o modelo treinado
    # nlp.to_disk(output_dir)

    return nlp


#### Exemplo de dados dos parâmetros:

In [ ]:
data_dict = entidades = {
    "BOMBA": [
    ("Bomba centrífuga", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de água", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de vácuo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de calor", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba dosadora", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba submersível", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de pistão", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de diafragma", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de engrenagem", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de incêndio", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba centrífuga", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de água", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de vácuo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de calor", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba dosadora", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba submersível", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de pistão", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de diafragma", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de engrenagem", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de incêndio", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de vácuo rotativa", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de alta pressão", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de transferência", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de ar comprimido", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de esgoto", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba peristáltica", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de irrigação", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de deslocamento positivo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de lóbulos", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de turbina", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de palhetas", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de cavidade progressiva", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba hidráulica", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de duplo diafragma", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de poço", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de drenagem", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de concreto", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de óleo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de químicos", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de refrigeração", {"entities": [(0, 5, "BOMBA")]})
    ]}

output_dir = r"..\documents"

iterations = 100
model = train_model(data_dict, iterations)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('..\documents\classes.xlsx')
df.head()